In [ ]:
import pymc3 as pm
import math
import numpy as np
import theano
import arviz as az
import pandas as pd

np.random.seed(42)
pm.set_tt_rng(42)

## Away Model and Data

In [ ]:
away_data = pd.read_csv("/home/morten/Develop/Live-Win-Prob/data/away.csv")
print(away_data.head())
print(away_data.shape)

In [ ]:
T_away = np.arange(100)
N_away = np.arange(300)
TN_away = np.empty((100, 300))
R_away = np.empty((100, 300))
X_away = np.empty((100, 8, 300))
t_away = TN_away.copy()

In [ ]:
## create X
for i, row in away_data.iterrows():
    if int(i / 100) < 300:
        for x in range(len(row)):
            if x == 0 or x == 9:
                continue
            X_away[int(row[0])][x - 1][int(i / 100)] = row[x]

# X = X.reshape(X.shape[0]*X.shape[1], X.shape[2])
X_away.shape

In [ ]:
## create t
for x in T_away:
    for y in N_away:
        t_away[x][y] = x

t_away.shape

In [ ]:
## create R
for i, row in away_data.iterrows():
    if int(i / 100) < 300:
        R_away[int(row[0])][int(i / 100)] = int(row[9])

R_away.shape

In [ ]:
with pm.Model() as away_model:
    at = pm.Normal("at", mu=0, sigma=math.sqrt(2), shape=(100, 8))
    beta = pm.Normal("beta", mu=0, sigma=math.sqrt(2))

    data = pm.Data("data", X_away)
    theta = pm.invlogit((at[:, :, None] * data).sum(axis=1) + beta)
    # theta = pm.invlogit(pm.math.dot(at, xt_theano) + beta)

    like = pm.Poisson("like", (100 - t_away) * theta, observed=R_away)

pm.model_to_graphviz(away_model)

In [ ]:
with away_model:
    away_trace = pm.sample(2000, return_inferencedata=True)

In [ ]:
with away_model:
    az.plot_trace(away_trace)

In [ ]:
# save away trace
with away_model:
    away_trace.to_netcdf("/home/morten/Develop/Live-Win-Prob/models/away_trace.nc")

## Home and Data

In [ ]:
home_data = pd.read_csv("/home/morten/Develop/Live-Win-Prob/data/home.csv")
print(home_data.head())
print(home_data.shape)

In [ ]:
T_home = np.arange(100)
N_home = np.arange(300)
TN_home = np.empty((100, 300))
R_home = np.empty((100, 300))
X_home = np.empty((100, 8, 300))
t_home = TN_home.copy()

In [ ]:
## create X
for i, row in home_data.iterrows():
    if int(i / 100) < 300:
        for x in range(len(row)):
            if x == 0 or x == 9:
                continue
            X_home[int(row[0])][x - 1][int(i / 100)] = row[x]

# X = X.reshape(X.shape[0]*X.shape[1], X.shape[2])
X_home.shape

In [ ]:
## create t
for x in T_home:
    for y in N_home:
        t_home[x][y] = x

t_home.shape

In [ ]:
## create R
for i, row in home_data.iterrows():
    if int(i / 100) < 300:
        R_home[int(row[0])][int(i / 100)] = int(row[9])

R_home.shape

In [ ]:
with pm.Model() as home_model:
    ha = pm.Normal("ha", mu=0, sigma=math.sqrt(2))
    at = pm.Normal("at", mu=0, sigma=math.sqrt(2), shape=(100, 8))
    beta = pm.Normal("beta", mu=0, sigma=math.sqrt(2))

    data = pm.Data("data", X_home)
    theta = pm.invlogit((at[:, :, None] * data).sum(axis=1) + beta + ha)
    # theta = pm.invlogit(pm.math.dot(at, xt_theano) + beta)

    like = pm.Poisson("like", (100 - t_home) * theta, observed=R_home)

pm.model_to_graphviz(home_model)

In [ ]:
with home_model:
    home_trace = pm.sample(2000, return_inferencedata=True)

In [ ]:
with home_model:
    az.plot_trace(home_trace)

In [ ]:
# save away trace
with home_model:
    home_trace.to_netcdf("/home/morten/Develop/Live-Win-Prob/models/home_trace.nc")